In [ ]:
import torch
from google.colab import files

# Save the model
torch.save(model.state_dict(), "best_resnet18.pth")

# Download the raw .pth file
files.download("best_resnet18.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
class IMG_DATA(Dataset):
    def __init__(self, df, transform=None):
        self.data = df
        self.transform = transform or transforms.ToTensor()
        self.label_map = {'NONPD': 0, 'PD': 1}

    def __getitem__(self, index):
        img_path = self.data.iloc[index].path
        label_str = self.data.iloc[index].label
        label = self.label_map[label_str]

        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long), img_path

    def __len__(self):
        return len(self.data)
test_dataset  = IMG_DATA(test_df, transform=transform)
test_loader   = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# --- Config ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 2
checkpoint_path = "best_resnet18.pth"

# --- Recreate model and load weights ---
model = ResNet18Classifier(num_classes=num_classes, dropout=0.3)
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.to(device)
model.eval()

# --- Label map (same as during training) ---
idx2label = {0: "NONPD", 1: "PD"}

# --- Inference on a single image ---
def predict_image(img_path: str):
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    img = Image.open(img_path).convert("RGB")
    tensor = preprocess(img).unsqueeze(0).to(device)  # add batch dim
    with torch.no_grad():
        logits = model(tensor)
        probs = torch.softmax(logits, dim=1)
        conf, pred = probs.max(dim=1)
    return idx2label[pred.item()], conf.item()

# Example:
label, confidence = predict_image("/content/dataset/PD/PD_SPECTROGRAM_1.png")
print(f"Predicted: {label} ({confidence*100:.1f}%)")


# --- Inference on a DataLoader (e.g., test_loader) ---
def evaluate_loader(loader):
    all_preds, all_confs = [], []
    model.eval()
    with torch.no_grad():
        # Changed to unpack the third item (img_path) as _)
        for images, _, _ in loader:
            images = images.to(device)
            logits = model(images)
            probs = torch.softmax(logits, dim=1)
            confs, preds = probs.max(dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_confs.extend(confs.cpu().tolist())
    # map back to labels
    labels = [idx2label[p] for p in all_preds]
    return labels, all_confs

# Example with your existing test_loader:
labels, confs = evaluate_loader(test_loader)
for i, (lbl, conf) in enumerate(zip(labels, confs)):
    print(f"{i:03d}: {lbl} ({conf*100:.1f}%)")

Predicted: PD (99.5%)
000: NONPD (99.7%)
001: PD (66.6%)
002: NONPD (99.2%)
003: PD (97.7%)
004: PD (99.3%)
005: NONPD (97.6%)
006: PD (98.9%)
007: NONPD (99.2%)
008: NONPD (99.7%)
009: NONPD (99.2%)
010: PD (98.6%)
011: PD (97.0%)
012: NONPD (99.1%)
013: NONPD (99.6%)


In [ ]:
with torch.no_grad():
    for images, labels, paths in test_loader:
        images = images.to(device)
        logits = model(images)
        probs = torch.softmax(logits, dim=1)
        confs, preds = probs.max(dim=1)

        # move back to CPU for printing
        preds = preds.cpu().tolist()
        confs = confs.cpu().tolist()

        for path, p, c in zip(paths, preds, confs):
            print(f"{path} → {idx2label[p]} ({c*100:.1f}%)")

/content/dataset/NONPD/NONPD_SPECTROGRAM_17.png → NONPD (99.7%)
/content/dataset/PD/PD_SPECTROGRAM_39.png → PD (66.6%)
/content/dataset/NONPD/NONPD_SPECTROGRAM_31.png → NONPD (99.2%)
/content/dataset/PD/PD_SPECTROGRAM_16.png → PD (97.7%)
/content/dataset/PD/PD_SPECTROGRAM_35.png → PD (99.3%)
/content/dataset/NONPD/NONPD_SPECTROGRAM_29.png → NONPD (97.6%)
/content/dataset/PD/PD_SPECTROGRAM_25.png → PD (98.9%)
/content/dataset/NONPD/NONPD_SPECTROGRAM_45.png → NONPD (99.2%)
/content/dataset/NONPD/NONPD_SPECTROGRAM_27.png → NONPD (99.7%)
/content/dataset/NONPD/NONPD_SPECTROGRAM_1.png → NONPD (99.2%)
/content/dataset/PD/PD_SPECTROGRAM_13.png → PD (98.6%)
/content/dataset/PD/PD_SPECTROGRAM_17.png → PD (97.0%)
/content/dataset/NONPD/NONPD_SPECTROGRAM_12.png → NONPD (99.1%)
/content/dataset/NONPD/NONPD_SPECTROGRAM_2.png → NONPD (99.6%)
